## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aklavik,CA,68.2191,-135.0107,49.89,57,100,5.06,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,39.47,99,100,1.28,light rain
2,2,Hermanus,ZA,-34.4187,19.2345,64.22,40,51,10.63,broken clouds
3,3,Santa Rosa,PH,14.3122,121.1114,82.11,78,78,3.20,broken clouds
4,4,Lasa,IT,46.6166,10.7002,59.88,84,5,5.06,clear sky
5,5,Catuday,PH,16.2923,119.8062,79.72,82,97,6.04,overcast clouds
6,6,Yellowknife,CA,62.4560,-114.3525,75.72,35,75,18.01,broken clouds
7,7,Djenne,ML,13.9061,-4.5533,98.42,12,6,4.12,clear sky
8,8,Ponta Do Sol,PT,32.6667,-17.1000,74.03,67,0,5.41,clear sky
9,9,Saskylakh,RU,71.9167,114.0833,41.94,70,0,17.09,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method

cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Santa Rosa,PH,14.3122,121.1114,82.11,78,78,3.20,broken clouds
5,5,Catuday,PH,16.2923,119.8062,79.72,82,97,6.04,overcast clouds
6,6,Yellowknife,CA,62.4560,-114.3525,75.72,35,75,18.01,broken clouds
7,7,Djenne,ML,13.9061,-4.5533,98.42,12,6,4.12,clear sky
16,16,Rikitea,PF,-23.1203,-134.9692,75.43,67,100,16.44,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows
print(len(cities_preferred_df))
cities_preferred_df.count()

224


City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

224


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Santa Rosa,PH,14.3122,121.1114,82.11,78,78,3.20,broken clouds
5,5,Catuday,PH,16.2923,119.8062,79.72,82,97,6.04,overcast clouds
6,6,Yellowknife,CA,62.4560,-114.3525,75.72,35,75,18.01,broken clouds
7,7,Djenne,ML,13.9061,-4.5533,98.42,12,6,4.12,clear sky
16,16,Rikitea,PF,-23.1203,-134.9692,75.43,67,100,16.44,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Santa Rosa,PH,82.11,broken clouds,14.3122,121.1114,
5,Catuday,PH,79.72,overcast clouds,16.2923,119.8062,
6,Yellowknife,CA,75.72,broken clouds,62.4560,-114.3525,
7,Djenne,ML,98.42,clear sky,13.9061,-4.5533,
16,Rikitea,PF,75.43,overcast clouds,-23.1203,-134.9692,
17,Tanout,NE,90.88,few clouds,14.9709,8.8879,
19,Puerto Ayora,EC,78.76,clear sky,-0.7393,-90.3518,
21,Maraba,BR,84.38,moderate rain,-5.3686,-49.1178,
22,Atuona,PF,77.61,few clouds,-9.8000,-139.0333,
23,Fortuna,US,79.93,few clouds,40.5982,-124.1573,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(url, params=params).json() 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
    
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Santa Rosa,PH,82.11,broken clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
5,Catuday,PH,79.72,overcast clouds,16.2923,119.8062,Benjie's Kubo & Cottages
6,Yellowknife,CA,75.72,broken clouds,62.4560,-114.3525,The Explorer Hotel
7,Djenne,ML,98.42,clear sky,13.9061,-4.5533,Hôtel Chez Baba
16,Rikitea,PF,75.43,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Puerto Ayora,EC,78.76,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Maraba,BR,84.38,moderate rain,-5.3686,-49.1178,Hotel Del Principe
22,Atuona,PF,77.61,few clouds,-9.8000,-139.0333,Villa Enata
23,Fortuna,US,79.93,few clouds,40.5982,-124.1573,Super 8 by Wyndham Fortuna
25,Olinda,BR,80.33,few clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))